In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('final.csv')
df.head(2)

,type,clock_cycle,thickness tFE,Vdd,T_RISE,T_FALL,T_DELAY
0,ncfet_inverter,100ps,0nm,0.3,2.410636e-12,2.278608e-12,4.401267e-12
1,ncfet_inverter,100ps,0nm,0.4,2.254149e-12,2.258426e-12,2.525869e-12


In [3]:
df.clock_cycle.unique()

array(['100ps', '200ps', '300ps', '400ps', '500ps', '50ps', '600ps',
       '700ps', '800ps', '900ps'], dtype=object)

In [4]:
#some cleanup of types for model
types = ['inverter', 'buffer', 'nand', 'nor']
type_dict = dict(zip(df.type.unique(), types))
df.type = df.type.map(type_dict)
df.clock_cycle = df.clock_cycle.map(lambda x: float(x[:-2]))
df['thickness tFE'] = df['thickness tFE'].map(lambda x: float(x[:-2]))
df.head(2)

,type,clock_cycle,thickness tFE,Vdd,T_RISE,T_FALL,T_DELAY
0,inverter,100.0,0.0,0.3,2.410636e-12,2.278608e-12,4.401267e-12
1,inverter,100.0,0.0,0.4,2.254149e-12,2.258426e-12,2.525869e-12


In [5]:
#why is this being dropped?
drop_50ps_idx = np.where(df['clock_cycle'] == 50)
df.drop(index=drop_50ps_idx[0], inplace=True)
df.shape

(6776, 7)

In [6]:
buffer_df = df[df.type == 'buffer']
inverter_df = df[df.type == 'inverter']
nor_df = df[df.type == 'nor']
nand_df = df[df.type == 'nand']
buffer_df.shape, inverter_df.shape, nand_df.shape, nor_df.shape

((1673, 7), (1701, 7), (1701, 7), (1701, 7))

In [7]:
def get_xy_df(df):
    X_buffer = np.vstack([df.clock_cycle, df['thickness tFE'], df['Vdd']]).T
    y_buffer = np.vstack([df['T_RISE '], df['T_FALL '], df['T_DELAY ']]).T
    return X_buffer, y_buffer

X_buffer, y_buffer = get_xy_df(buffer_df)
X_inverter, y_inverter = get_xy_df(inverter_df)
X_nor, y_nor = get_xy_df(nor_df)
X_nand, y_nand = get_xy_df(nand_df)
X_buffer.shape, y_buffer.shape

((1673, 3), (1673, 3))

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [16]:
def ridge_reg(x_train, y_train, x_valid, y_valid):
    print('Doing Ridge Regression.....')
    ridge = Ridge(alpha = 1)
    ridge.fit(x_train, y_train)
    print('training score = ', ridge.score(x_train, y_train))
    y_pred_ridge = ridge.predict(x_valid)
    mse_ridge = mean_squared_error(y_valid, y_pred_ridge) 
    print('test score =', ridge.score(x_valid, y_valid)) 

def do_rf(x_train, y_train, x_valid, y_valid):
    print('\n\ndoing RF.....')
    RF = RandomForestRegressor(max_depth=100, random_state=0)
    RF.fit(x_train, y_train)
    print('trainig score = ', RF.score(x_train, y_train))
    y_pred = RF.predict(x_valid)
    mse = mean_squared_error(y_valid, y_pred)
    r2 = r2_score(y_valid, y_pred) # coefficient of determination
    print('test score =', RF.score(x_valid, y_valid))
    print(f'the mse is {mse}')
    print(f"the r2 score is: {r2}")
    return y_pred

def train_models(x, y, random_state=0):
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, 
                                                        random_state=random_state, 
                                                        test_size=0.1)    
    ridge_reg(x_train, y_train, x_valid, y_valid)
    y_pred = do_rf(x_train, y_train, x_valid, y_valid)
    return y_pred

### Lets start with our Buffer circuit

In [10]:
_ = train_models(X_buffer, y_buffer, random_state=123)

Doing Ridge Regression.....
training score =  0.9167102409624371
test score = 0.8967879854390292


doing RF.....
trainig score =  -3.6984307936194227e-06
test score = -0.0005074970005255105
the mse is 4.938222597074191e-23


### Inverter

In [167]:
_ = train_models(X_inverter, y_inverter, random_state=0)

Doing Ridge Regression.....
training score =  0.9068440076824577
test score = 0.9137210104799065


doing RF.....
trainig score =  -8.346795311882834e-06
test score = -0.01963368652833042
the mse is 4.9345836306404944e-23


### Nand

In [168]:
_ = train_models(X_nand, y_nand, random_state=1)

Doing Ridge Regression.....
training score =  0.8901950211342888
test score = 0.8801793859431911


doing RF.....
trainig score =  -8.62405818091716e-06
test score = -0.0011959744121536147
the mse is 4.134560228860396e-23


### Nor

In [154]:
_ = train_models(X_nor, y_nor)

Doing Ridge Regression.....
training score =  0.8899918684137367
test score = 0.898750119906659


doing RF.....
trainig score =  -9.065795662843712e-06
test score = -0.018704305257992344
the mse is 3.088929916154021e-23


# Normalizing the dependent variable helps?

Looks like things are not working as expected out of the box! Normally you do not need to do much with the data with a random-forest-classifier. but apparently this is not the case with regressors.....

In [17]:
_ = train_models(X_nor, y_nor/y_nor.min())

Doing Ridge Regression.....
training score =  0.889991868413737
test score = 0.8987501199066591


doing RF.....
trainig score =  0.9994965217624411
test score = 0.997362112592557
the mse is 0.001056661204020164
the r2 score is: 0.997362112592557


So by normalizing the dependent values we can replicate the previous Random Forest Model perforamnce....

In [18]:
_ = train_models(X_buffer, y_buffer/y_buffer.min())

Doing Ridge Regression.....
training score =  0.9151547232450441
test score = 0.9096239937163108


doing RF.....
trainig score =  0.9996718330442516
test score = 0.9981174754219625
the mse is 0.06108506176995491
the r2 score is: 0.9981174754219625


In [19]:
_ = train_models(X_inverter, y_inverter / y_inverter.min())

Doing Ridge Regression.....
training score =  0.9068440076824578
test score = 0.9137210104799065


doing RF.....
trainig score =  0.9995370899090714
test score = 0.9973107488297214
the mse is 0.18724402530283232
the r2 score is: 0.9973107488297214


In [20]:
_ = train_models(X_nand, y_nand / y_nand.min())

Doing Ridge Regression.....
training score =  0.8869122400572241
test score = 0.9092443535388508


doing RF.....
trainig score =  0.9994631916438235
test score = 0.9970597571271802
the mse is 0.0711064052789472
the r2 score is: 0.9970597571271802
